In [ ]:
import firedrake
from firedrake import Constant, inner, sym, grad, div, dx
from ufl.algorithms import extract_coefficients
import matplotlib.pyplot as plt

In [ ]:
ε = lambda u: sym(grad(u))

def get_test_function(u):
    z, = extract_coefficients(u)
    Z = z.function_space()
    w = firedrake.TestFunction(Z)
    return firedrake.replace(u, {z: w})

In [ ]:
def form_viscous_dissipation(**kwargs):
    u = kwargs["velocity"]
    μ = kwargs["viscosity"]
    
    v = get_test_function(u)
    
    return 2 * μ * inner(ε(u), ε(v)) * dx


def form_pressure_constraint(**kwargs):
    u = kwargs["velocity"]
    p = kwargs["pressure"]

    v = get_test_function(u)
    q = get_test_function(p)

    return -(p * div(v) + q * div(u)) * dx

In [ ]:
nx = 32
mesh = firedrake.UnitSquareMesh(nx, nx, diagonal="crossed")

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_axis_off()
colors = ["tab:blue", "tab:orange", "tab:purple", "tab:green"]
firedrake.triplot(mesh, axes=ax, boundary_kw={"colors": colors})
ax.legend(loc="upper right");

In [ ]:
cg1 = firedrake.FiniteElement("CG", "triangle", 1)
cg2 = firedrake.FiniteElement("CG", "triangle", 2)

Q = firedrake.FunctionSpace(mesh, cg1)
V = firedrake.VectorFunctionSpace(mesh, cg2)

Z = V * Q
z = firedrake.Function(Z)

In [ ]:
u, p = firedrake.split(z)

μ = Constant(1.0)

fields = {"velocity": u, "pressure": p}
parameters = {"viscosity": μ}

fns = [form_viscous_dissipation, form_pressure_constraint]
F = sum([fn(**fields, **parameters) for fn in fns])

In [ ]:
bcs = [
    firedrake.DirichletBC(Z.sub(0), 0, [1, 2, 3]),
    firedrake.DirichletBC(Z.sub(0), Constant((1.0, 0.0)), [4]),
]

In [ ]:
firedrake.solve(F == 0, z, bcs)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_axis_off()
arrows = firedrake.quiver(u, axes=ax)
fig.colorbar(arrows);